# 2026 ASA Prediction Competition: HDL Cholesterol Prediction
**Team Members:** Matheus Gomes, Satvik Hulikere, Joaquin Hidalgo, Caio Rocha  
**Objective:** To develop a robust machine learning pipeline to predict Direct HDL-Cholesterol levels using dietary and demographic data from the NHANES dataset.



In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# Reading CSV training file
df = pd.read_csv("/train.csv")

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/train.csv'

In [ ]:
# Checking missing values
df.info()

# Checking duplicate values
df = df.drop_duplicates()

## 🛠️ Data Preprocessing & Feature Engineering
To improve model performance and interpretability, we perform the following steps:
* **Renaming:** Converting technical NHANES codes (e.g., `DR1TKCAL`) into human-readable labels using the provided variable mapping.
* **Feature Removal:** We drop administrative metadata (Interviewer IDs, Sample Weights) and logistical variables that do not carry physiological significance for cholesterol prediction.
* **Duplicate Handling:** Ensuring data integrity by removing any redundant entries.

In [ ]:
# List of columns to be removed for now (id or interview info variables)
cols_to_drop = [
    'Unnamed: 0', 'DR1MRESP', 'DR1HELP',
    'DR1LANG', 'DR1EXMER'
]

# Dropping the columns directly from the main dataframe
df.drop(columns=cols_to_drop, inplace=True)

# Verify the result
print(f"Current number of columns: {len(df.columns)}")
df.head()

In [ ]:
# 1. Load the variable labels
labels_df = pd.read_csv("/variable_labels.csv")

# 2. Create a dictionary mapping: {'Variable_Name': 'Human_Readable_Label'}
# This assumes the CSV has 'variable' and 'label' columns
label_dict = dict(zip(labels_df['variable'], labels_df['label']))

# 3. Rename the columns in your main dataframe
df.rename(columns=label_dict, inplace=True)

# 4. View the result with the new readable names
print("Columns renamed successfully!")
df.head()

## 🔍 Feature Selection: Variance & Multicollinearity
We apply statistical filters to streamline the feature set:
1. **Low Variance Filter:** Removing features with near-zero variance that offer no predictive power.
2. **Correlation Analysis:** Identifying highly correlated features ($r > 0.90$). We remove redundant variables (like specific Fatty Acid sub-types) to prevent **Multicollinearity**, which can destabilize linear model coefficients.

START FILTERING UNNECESSARY COLUMNS/VARIABLES

In [ ]:
# sample variance from each column
variance = df.var()

# store low variance variables in a list
low_variance_cols = [col for col, var in variance.items() if var < 0.005]

# identify variables to be removed based on their variance
print("Columns with low variance: ", low_variance_cols)

# look further into these variables
df[['PFA 18:4 (Octadecatetraenoic) (gm)', 'Breast-fed infant (either day)', 'Ever had a drink of any kind of alcohol']].head(10)

# print correlation from each low variance column against the target
low_var_corr = [df[col].corr(df['Direct HDL-Cholesterol (mg/dL)'])
 for col in low_variance_cols]
print(low_var_corr)

# drop low variance and uncorrelated columns
df = df.drop(columns=['Breast-fed infant (either day)', 'Ever had a drink of any kind of alcohol'])

print(f"Current number of columns: {len(df.columns)}")

/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]

  This tells us standard deviation is 0. Remove these columns. Keep 'PFA 18:4 (Octadecatetraenoic) (gm)' despite low linear correlation

perform feature to feature correlation analysis. Remove highly correlated features to prevent multicollinearity later on.

In [ ]:
# keep removing based on feature to feature correlation

# separate features and target
target_col = 'Direct HDL-Cholesterol (mg/dL)'

y = df[target_col]
X = df.drop(columns=[target_col])

# obtain correlation among predictors only
corr_matrix = X.corr().abs()

# only obtain upper triangle to avoid duplicate comparisons
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# identify highly correlated features (>0.9)
to_drop = [column for column in upper.columns
           if any(upper[column] > 0.9)]
print(to_drop)

# drop highly correlated features
df = df.drop(columns=to_drop)

print(f"Current number of columns: {len(df.columns)}")

We will not be removing variables based on weak correlation against the target variable due to that they show linear relationship. However, we will be using regularized linear regression for prediction as well as tree based models which dont rely on this. We will use this kind of models due to the high amount of variables we have, we must avoid overfitting.

STEPS TO FOLLOW:

5️⃣ Baseline Model

Train a simple model first:

Linear Option

LASSO, Ridge, or ElasticNet

Tree Option

Random Forest or XGBoost

Evaluate performance (RMSE, R²) on validation set.

6️⃣ Feature Importance / Refinement

For tree models, check feature_importances_

Remove weak features or try feature grouping if needed

In [ ]:
# check for outliers and plausible values

df.describe()

In [ ]:
df.info()

## 🤖 Model Development & Evaluation
We evaluate multiple algorithms to balance simplicity and predictive power. Our primary metric is **Root Mean Squared Error (RMSE)**.
* **Lasso Regression:** Used as a baseline to perform automated feature selection through L1 regularization.
* **Random Forest:** A non-linear ensemble approach to capture complex interactions.
* **XGBoost:** Our final optimized model, utilizing gradient boosting with tuned hyperparameters to minimize the prediction gap.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

# 1. Define Features and Target
X = df.drop(columns=['Direct HDL-Cholesterol (mg/dL)'])
y = df['Direct HDL-Cholesterol (mg/dL)']

# 2. Split into Training and Validation sets (80/20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Scale the data (Crucial for Lasso!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
from sklearn.linear_model import LassoCV

# 1. Initialize LassoCV
# We provide a range of alphas to test (from 0.001 to 10)
alphas = np.logspace(-2, 2, 100)
lasso_cv = LassoCV(alphas=alphas, cv=10, max_iter=20000, random_state=42)

# 2. Fit the model on scaled training data
lasso_cv.fit(X_train_scaled, y_train)

# 3. Best Alpha found
print(f"Optimal Alpha: {lasso_cv.alpha_:.6f}")

# 4. Evaluate on validation set
# Training predictions
y_train_pred = lasso_cv.predict(X_train_scaled)

# Training metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

y_cv_pred = lasso_cv.predict(X_val_scaled)

cv_rmse = np.sqrt(mean_squared_error(y_val, y_cv_pred))
cv_r2 = r2_score(y_val, y_cv_pred)

print(f"Training RMSE: {train_rmse:.4f}")
print(f"Training R²: {train_r2:.4f}")

# Validation metrics (already computed)
print(f"Validation RMSE: {cv_rmse:.4f}")
print(f"Validation R²: {cv_r2:.4f}")

Optimal Alpha: 0.236449
Training RMSE: 5.4001
Training R²: 0.6362
Validation RMSE: 5.9078
Validation R²: 0.5813

Model including highly correlated features slightly outperformed old model. Keep all features.

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=42) # we shuffle for cv to be unbiased

# Alpha grid (Ridge typically prefers slightly larger values than Lasso)
alphas = np.logspace(-2, 2, 100)

# Initialize RidgeCV
ridge_cv = RidgeCV(
    alphas=alphas,
    cv=kf
)

# Fit on scaled training data
ridge_cv.fit(X_train_scaled, y_train)

# Best alpha
print(f"Optimal Alpha: {ridge_cv.alpha_:.6f}")

# ---- Training predictions ----
y_train_pred = ridge_cv.predict(X_train_scaled)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

# ---- Validation predictions ----
y_cv_pred = ridge_cv.predict(X_val_scaled)

cv_rmse = np.sqrt(mean_squared_error(y_val, y_cv_pred))
cv_r2 = r2_score(y_val, y_cv_pred)

# ---- Print results ----
print(f"Training RMSE: {train_rmse:.4f}")
print(f"Training R²: {train_r2:.4f}")

print(f"Validation RMSE: {cv_rmse:.4f}")
print(f"Validation R²: {cv_r2:.4f}")

In [ ]:
from sklearn.linear_model import ElasticNetCV

# L1 ratio grid: 0=L2 (Ridge), 1=L1 (Lasso), 0.5=balanced
l1_ratios = [0.9, 0.95, 0.98, 0.99] # lean more to lasso side since its performing better

# Initialize ElasticNetCV
elastic_cv = ElasticNetCV(
    alphas=alphas,
    l1_ratio=l1_ratios,
    cv=10,
    max_iter=50000,
    tol=1e-3,
    n_jobs=-1,
    random_state=42
)

# Fit model on scaled training data
elastic_cv.fit(X_train_scaled, y_train)

# Best alpha & l1_ratio
print(f"Optimal Alpha: {elastic_cv.alpha_:.6f}")
print(f"Optimal L1 Ratio: {elastic_cv.l1_ratio_:.2f}")

# ---- Training predictions ----
y_train_pred = elastic_cv.predict(X_train_scaled)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

# ---- Validation predictions ----
y_cv_pred = elastic_cv.predict(X_val_scaled)
cv_rmse = np.sqrt(mean_squared_error(y_val, y_cv_pred))
cv_r2 = r2_score(y_val, y_cv_pred)

# ---- Print results ----
print(f"Training RMSE: {train_rmse:.4f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Validation RMSE: {cv_rmse:.4f}")
print(f"Validation R²: {cv_r2:.4f}")

Conlcusion: Lasso is the better choice at this point

check for distributions as a last resort

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histogram of target
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(y_train, bins=30, kde=True, color='skyblue')
plt.title("Target Distribution - Histogram")
plt.xlabel("y_train")
plt.ylabel("Frequency")

# Boxplot of target
plt.subplot(1, 2, 2)
sns.boxplot(x=y_train, color='lightgreen')
plt.title("Target Distribution - Boxplot")
plt.xlabel("y_train")

plt.tight_layout()
plt.show()

We will not be adding interactions in the linear models to keep them simple. We will explore non linear relationships later with tree based models. Afterwards, we are considering stacking the two type of models to get model diversity.

Random Forest

In [ ]:
'''from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define a focused search space
param_dist = {
    'n_estimators': [200, 300, 500],
    'max_depth': [8, 10, 12],                # Testing around your current 10
    'min_samples_split': [5, 10, 15],        # Higher helps reduce overfitting
    'min_samples_leaf': [2, 4, 6],           # Key for smoothing out predictions
    'max_features': ['sqrt', 'log2', 0.3],   # Limits features per tree (vital for 77 features)
    'bootstrap': [True]
}

rf_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1),
    param_distributions=param_dist,
    n_iter=20,
    cv=10,                                   # Use 10-fold as we discussed
    scoring='neg_root_mean_squared_error',
    verbose=1,
    random_state=42
)

# Train on ORIGINAL features
rf_search.fit(X_train, y_train)

print(f"Best RF Params: {rf_search.best_params_}")
print(f"Best CV RMSE: {-rf_search.best_score_:.4f}") '''

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 1. Initialize the Random Forest with the optimized parameters from your CV
rf_model_optimized = RandomForestRegressor(
    n_estimators=600,      # Increased from 100 based on CV
    max_depth=10,          # Added based on CV to prevent overfitting
    min_samples_split=5,  # Added based on CV for better generalization
    min_samples_leaf=2,
    max_features=0.5,     # CV found that looking at all 77 features worked best
    random_state=42,
    n_jobs=-1
)

# 2. Fit the model using your scaled training data
rf_model_optimized.fit(X_train, y_train)

# 3. Make predictions on the validation set
y_rf_pred = rf_model_optimized.predict(X_val)

# 4. Evaluate Performance
rf_rmse = np.sqrt(mean_squared_error(y_val, y_rf_pred))
rf_r2 = r2_score(y_val, y_rf_pred)

print(f"Optimized Random Forest RMSE: {rf_rmse:.4f}")
print(f"Optimized Random Forest R²: {rf_r2:.4f}")

# 1. Get predictions for both sets (using SCALED data as per your RF training)
y_rf_train_pred = rf_model_optimized.predict(X_train)
y_rf_val_pred = rf_model_optimized.predict(X_val)

# 2. Calculate RMSE
rf_train_rmse = np.sqrt(mean_squared_error(y_train, y_rf_train_pred))
rf_val_rmse = np.sqrt(mean_squared_error(y_val, y_rf_val_pred))

print(f"--- Random Forest Performance ---")
print(f"Train RMSE: {rf_train_rmse:.4f}")
print(f"Validation RMSE: {rf_val_rmse:.4f}")
print(f"Difference (Gap): {rf_val_rmse - rf_train_rmse:.4f}")

Best performance with optimal parameters:

Optimized Random Forest RMSE: 5.1240
Optimized Random Forest R²: 0.6850
--- Random Forest Performance ---
Train RMSE: 2.1833
Validation RMSE: 5.1240
Difference (Gap): 2.9407


Looks like tree is slightly overfitted. Find optimal parameters by testing:
- tree depth
- leave size
- features consdidered each split

In [ ]:
'''# Find optimal parameters for RF

from sklearn.model_selection import RandomizedSearchCV

# Base model
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Parameter grid
param_dist = {
    "n_estimators": [400],
    "max_depth": [6, 8, 10],   # dont let tree get too deep to avoid overfitting
    "min_samples_split": [6, 8, 10],
    "min_samples_leaf": [3, 6, 9],
    "max_features": ["sqrt", 0.5]
}

# Randomized Search
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=40,               # number of random combinations
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# fit model on training data
rf_random.fit(X_train, y_train)

print("Best Parameters:", rf_random.best_params_)
print("Best CV RMSE:", -rf_random.best_score_)

# Use optimal parameters
rf_best = rf_random.best_estimator_

y_train_pred = rf_best.predict(X_train)
y_val_pred = rf_best.predict(X_val)

# Validate model on training and testing data (RMSE)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

# (R^2)
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Gap: {val_rmse - train_rmse:.4f}")
print(f"Train R²: {train_r2:.4f}")
print(f"Validation R²: {val_r2:.4f}")'''

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best Parameters: {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 0.5, 'max_depth': 10}
Best CV RMSE: 4.826052255457246
Train RMSE: 2.1911
Validation RMSE: 5.1336
Gap: 2.9425
Train R²: 0.9401
Validation R²: 0.6839

optimal parameters for now. test a few more

XGBoost

now we run it again, but this time in crease the reg_lambda which forces the model to dsitrubute importance more effecitvely rather than over-relying on one.

In [ ]:
'''import xgboost as xgb

# Use the best parameters found earlier, but increase regularization
xgb_lean = xgb.XGBRegressor(
    n_estimators=1000,      # Increase trees...
    learning_rate=0.01,     # ...but slow down learning for precision
    max_depth=5,            # Keep it shallow to prevent overfitting
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=20,          # Increase L2 to handle the BMI/Waist correlation
    reg_alpha=1,            # Added L1 to further prune remaining noise
    random_state=42,
    n_jobs=-1
)

# Fit with Early Stopping to find the exact moment the model peaks
xgb_lean.fit(
    X_train_lean, y_train,
    eval_set=[(X_val_lean, y_val)],
    verbose=False
)

# Evaluate
y_lean_pred = xgb_lean.predict(X_val_lean)
lean_rmse = np.sqrt(mean_squared_error(y_val, y_lean_pred))
lean_r2 = r2_score(y_val, y_lean_pred)

print(f"Lean XGBoost RMSE: {lean_rmse:.4f}")
print(f"Lean XGBoost R²: {lean_r2:.4f}")'''

In [ ]:
'''
# test parameters for xgboost to find best performance
import xgboost as xgb
from scipy.stats import uniform, randint

xgb_model = xgb.XGBRegressor(
    n_estimators=1000,
    random_state=42,
    n_jobs=-1
)

param_dist = {
    'learning_rate': uniform(0.01, 0.09),      # 0.01–0.1
    'max_depth': randint(3, 8),               # 3–7
    'min_child_weight': randint(1, 6),        # 1–5
    'subsample': uniform(0.6, 0.3),           # 0.6–0.9
    'colsample_bytree': uniform(0.6, 0.3),    # 0.6–0.9
    'gamma': uniform(0, 5),                   # 0–5
    'reg_alpha': uniform(0, 5),               # 0–5
    'reg_lambda': uniform(1, 10),             # 1–11
}

xgb_random = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=50,              # number of random combinations
    scoring='neg_root_mean_squared_error',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

xgb_random.fit(X_train, y_train)
print("Best Parameters:", xgb_random.best_params_)
print("Best CV RMSE:", -xgb_random.best_score_)'''

In [ ]:
'''
# 4. Use the best parameters and add early stopping
best_params = xgb_random.best_params_

best_xgb = xgb.XGBRegressor(
    n_estimators=10000,      # large number, early stopping will pick the optimal
    **best_params,           # unpack all the tuned parameters
    random_state=42,
    n_jobs=-1
)

best_xgb.fit(X_train, y_train)

# 7️⃣ Predictions
y_train_pred = best_xgb.predict(X_train)
y_val_pred = best_xgb.predict(X_val)

# 8️⃣ Evaluate performance
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Train RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}")
print(f"Val RMSE: {val_rmse:.4f}, R²: {val_r2:.4f}")
print(f"RMSE Gap (Val - Train): {val_rmse - train_rmse:.4f}")'''

Regular Cross-Validated xgboost

In [ ]:
import xgboost as xgb

# 1. Initialize the XGBoost Regressor
# We'll start with some robust default 'learning_rate' and 'max_depth'
xgb_model = xgb.XGBRegressor(
    n_estimators=1000,     # More trees than RF, but they are 'shallower'
    learning_rate=0.01,   # 'eta' - smaller steps prevent overfitting
    max_depth=3,          # Typical range 3-10
    subsample=0.8,        # Use 80% of data for each tree to add randomness
    colsample_bytree=0.8, # Similar to max_features in RF
    random_state=42,
    reg_lambda = 10,
    reg_alpha = 1,
    objective='reg:squarederror' # Explicitly set objective for clarity
)

# Define the stopping logic separately
early_stop = xgb.callback.EarlyStopping(
    rounds=50,
    metric_name='rmse',
    data_name='validation_0' # Refers to the first set in eval_set
)

# Now your .fit() looks almost like it used to
xgb_model.fit(
    X_train, y_train,
 # This replaces the old early_stopping_rounds=50
)

# 3. Make predictions
y_xgb_pred = xgb_model.predict(X_val)

# 4. Evaluate Performance
xgb_rmse = np.sqrt(mean_squared_error(y_val, y_xgb_pred))
xgb_r2 = r2_score(y_val, y_xgb_pred)

print(f"XGBoost RMSE: {xgb_rmse:.4f}")
print(f"XGBoost R²: {xgb_r2:.4f}")

# 1. Get predictions for both sets
y_train_pred = xgb_model.predict(X_train)
y_val_pred = xgb_model.predict(X_val)

# 2. Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f"--- XGBoost Performance ---")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Difference (Gap): {val_rmse - train_rmse:.4f}")

In [ ]:
'''from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Define the search space
param_dist = {
    'n_estimators': [500, 1000, 2000],          # High number, let early stopping handle it
    'learning_rate': [0.01, 0.05, 0.1],         # Smaller is usually better but slower
    'max_depth': [3, 5, 7, 9],                  # Start shallow (3-5) to avoid overfit
    'min_child_weight': [1, 5, 10],             # Higher values prevent small, noisy leaves
    'subsample': [0.7, 0.8, 0.9],               # Row sampling
    'colsample_bytree': [0.6, 0.7, 0.8],        # Feature sampling (crucial for 77 features)
    'reg_alpha': [0, 0.1, 1, 10],               # L1 (Lasso-style)
    'reg_lambda': [1, 5, 10]                    # L2 (Ridge-style)
}

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)

# Run Random Search
random_search = RandomizedSearchCV(
    xgb_reg,
    param_distributions=param_dist,
    n_iter=50,             # Number of random combinations to try
    scoring='neg_root_mean_squared_error',
    cv=5,
    verbose=1
)

# Remember: No scaling needed for XGBoost!
random_search.fit(X_train, y_train)

print(f"Best Params: {random_search.best_params_}")
print(f"Best Score: {-random_search.best_score_}")'''

Stacking (Random Forest + Lasso)

In [ ]:
# Use split data from before

lasso = Lasso(alpha=0.1, max_iter = 10000)
rf = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)

from sklearn.model_selection import KFold, cross_val_predict

kf = KFold(n_splits=10, shuffle=True, random_state=42)

lasso_oof = cross_val_predict(lasso, X_train, y_train, cv=kf)
rf_oof = cross_val_predict(rf, X_train, y_train, cv=kf)

# Stack predictions as new features
X_meta_train = np.column_stack([lasso_oof, rf_oof])

from sklearn.linear_model import Ridge
meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta_train, y_train)

# Fit base models on full training data
lasso.fit(X_train_scaled, y_train)
rf.fit(X_train, y_train)

lasso_val_pred = lasso.predict(X_val_scaled)
rf_val_pred = rf.predict(X_val)

X_meta_val = np.column_stack([lasso_val_pred, rf_val_pred])

# Final stacked predictions
stack_val_pred = meta_model.predict(X_meta_val)




In [ ]:
def eval_reg(y_true, y_pred, name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{name:>10} | RMSE = {rmse:.4f} | R^2 = {r2:.4f}")

eval_reg(y_val, stack_val_pred, "Stacked")

Stacking with Lasso, random forest, and xgboost

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Define the base models using your previous best parameters
base_models = [
    ('lasso', lasso_cv), # Uses the optimized alpha you found
    ('rf', rf_model_optimized), # The 200-tree model from cell 89
    ('xgb', xgb_model) # The optimized XGBoost from your search
]

# 2. Initialize the Stacking Regressor
# The final_estimator 'Ridge' will decide how much to weight each model's prediction
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge(alpha=1.0),
    cv=5,
    n_jobs=-1
)

# 3. Fit the stack
# This will take a moment as it does cross-validation internally
stacking_model.fit(X_train, y_train)

# 4. Predict and Evaluate
y_stack_pred = stacking_model.predict(X_val)
stack_rmse = np.sqrt(mean_squared_error(y_val, y_stack_pred))
stack_r2 = r2_score(y_val, y_stack_pred)

print(f"Stacking Model RMSE: {stack_rmse:.4f}")
print(f"Stacking Model R²: {stack_r2:.4f}")


# 1. Generate predictions for both sets using the Triple Stack
# This uses the model from Cell 39
y_stack_train_pred = stacking_model.predict(X_train)
y_stack_val_pred = stacking_model.predict(X_val)

# 2. Calculate RMSE
stack_train_rmse = np.sqrt(mean_squared_error(y_train, y_stack_train_pred))
stack_val_rmse = np.sqrt(mean_squared_error(y_val, y_stack_val_pred))

# 3. Calculate R² for the full picture
stack_train_r2 = r2_score(y_train, y_stack_train_pred)
stack_val_r2 = r2_score(y_val, y_stack_val_pred)

print(f"--- Triple Stack (Lasso + RF + XGB) Performance ---")
print(f"Train RMSE: {stack_train_rmse:.4f} | Train R²: {stack_train_r2:.4f}")
print(f"Validation RMSE: {stack_val_rmse:.4f} | Validation R²: {stack_val_r2:.4f}")
print(f"Gap (Overfitting): {stack_val_rmse - stack_train_rmse:.4f}")

Stacking Lasso and xgboost

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# 1. Define the base models (Lasso and XGBoost only)
base_models_lean = [
    ('lasso', lasso_cv),
    ('xgb', xgb_model)
]

# 2. Initialize the Stacking Regressor
# We keep Ridge as the final estimator to prevent the meta-model from overfitting
stack_lasso_xgb = StackingRegressor(
    estimators=base_models_lean,
    final_estimator=Ridge(alpha=1.0),
    cv=5,
    n_jobs=-1
)

# 3. Fit the model
stack_lasso_xgb.fit(X_train_scaled, y_train)

# 4. Predict and Evaluate
y_stack_lean_pred = stack_lasso_xgb.predict(X_val_scaled)
stack_lean_rmse = np.sqrt(mean_squared_error(y_val, y_stack_lean_pred))
stack_lean_r2 = r2_score(y_val, y_stack_lean_pred)

print(f"Lasso + XGBoost Stack RMSE: {stack_lean_rmse:.4f}")
print(f"Lasso + XGBoost Stack R²: {stack_lean_r2:.4f}")

Stacking xgboost and randomforest

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Define the base models (RF and XGBoost only)
base_models_trees = [
    ('rf', rf_model_optimized),
    ('xgb', xgb_model)
]

# 2. Initialize the Stacking Regressor
stack_trees = StackingRegressor(
    estimators=base_models_trees,
    final_estimator=Ridge(alpha=1.0),
    cv=5,
    n_jobs=-1
)

# 3. Fit the model
stack_trees.fit(X_train, y_train)

# 4. Predict and Evaluate
y_stack_trees_pred = stack_trees.predict(X_val)
stack_trees_rmse = np.sqrt(mean_squared_error(y_val, y_stack_trees_pred))
stack_trees_r2 = r2_score(y_val, y_stack_trees_pred)

print(f"RF + XGBoost Stack RMSE: {stack_trees_rmse:.4f}")
print(f"RF + XGBoost Stack R²: {stack_trees_r2:.4f}")

# 1. Generate predictions for both sets
# stack_trees is your StackingRegressor from Cell 54
y_stack_train_pred = stack_trees.predict(X_train)
y_stack_val_pred = stack_trees.predict(X_val)

# 2. Calculate RMSE
stack_train_rmse = np.sqrt(mean_squared_error(y_train, y_stack_train_pred))
stack_val_rmse = np.sqrt(mean_squared_error(y_val, y_stack_val_pred))

print(f"--- Stacking (RF + XGBoost) Performance ---")
print(f"Train RMSE: {stack_train_rmse:.4f}")
print(f"Validation RMSE: {stack_val_rmse:.4f}")
print(f"Gap: {stack_val_rmse - stack_train_rmse:.4f}")

Model Performance Comparison

Model Combination,RMSE (Lower is Better),R² (Higher is Better)
RF + XGBoost (Winner),5.0156,0.6982
Lasso + Random Forest,5.0436,0.6948
Lasso + RF + XGBoost,5.0465,0.6945
Lasso + XGBoost,5.0470,0.6944

Actual Vs Predicted Graph for Lasso and Random FOrest

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plotting the three models for comparison
plt.scatter(y_val, lasso_val_pred, alpha=0.5, label=f'Lasso (RMSE: 5.91)', color='blue')
plt.scatter(y_val, rf_val_pred, alpha=0.5, label=f'RF (RMSE: 5.15)', color='green')
plt.scatter(y_val, stack_val_pred, alpha=0.6, label=f'Stacked (RMSE: 5.04)', color='red', edgecolors='k')

# Identity line (Perfect Prediction)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)

plt.xlabel('Actual HDL-Cholesterol')
plt.ylabel('Predicted HDL-Cholesterol')
plt.title('Actual vs. Predicted: Base Models vs. Stacked Ensemble')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# 1. Base Model: Lasso (Scaled)
plt.scatter(y_val, lasso_val_pred, alpha=0.4, label=f'Lasso Base (RMSE: 5.91)', color='#3498db')

# 2. Base Model: Random Forest (Original)
plt.scatter(y_val, rf_val_pred, alpha=0.4, label=f'RF Base (RMSE: 5.15)', color='#2ecc71')

# 3. The "Master" Model: Stacked Ensemble
plt.scatter(y_val, stack_val_pred, alpha=0.7, label=f'Stacked Ensemble (RMSE: 5.04)',
            color='#e74c3c', edgecolors='white', linewidth=0.5)

# Identity Line
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'k--', lw=2, label='Perfect Prediction')

plt.title('Actual vs. Predicted: How Stacking Combines Lasso & RF')
plt.xlabel('Actual HDL-Cholesterol')
plt.ylabel('Predicted HDL-Cholesterol')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

## 📤 Final Predictions
After identifying the **3-Stack** as our best-performing model (lowest Validation RMSE), we apply the same preprocessing pipeline to the test set and generate the final predictions for submission.

In [ ]:
# 1. Load test data
df_test = pd.read_csv("/test.csv")

# 2. Apply the same Human Readable labels used in training
df_test.rename(columns=label_dict, inplace=True)

# 3. Filter for ONLY the features your model was trained on
# model_features = X_train.columns.tolist()
X_test_final = df_test[X_train.columns]

# 4. Handle any missing values in the test set (Random Forest requirement)
X_test_final = X_test_final.fillna(X_train.median())

In [ ]:
# Generate predictions using the 3-model stack
# Make sure 'stacking_model' is the variable name for your Lasso+RF+XGB stack
test_predictions_triple = stacking_model.predict(X_test_final)

# Create the submission file
submission = pd.DataFrame({
    'Id': df_test.iloc[:, 0], # Grabs the unique ID from the first column
    'LBDHDD_outcome': test_predictions_triple
})

# Save to CSV
submission.to_csv('pred.csv', index=False)
print("Submission file 'pred.csv' is ready!")

Testing pred.csv

In [ ]:
# Load your newly created predictions
preds_df = pd.read_csv("pred.csv")

print("--- Statistical Comparison ---")
print(f"Train Actual Mean: {y_train.mean():.2f}")
print(f"Test Prediction Mean: {preds_df['LBDHDD_outcome'].mean():.2f}")
print("\n--- Value Range ---")
print(f"Train Range: {y_train.min():.2f} to {y_train.max():.2f}")
print(f"Test Range: {preds_df['LBDHDD_outcome'].min():.2f} to {preds_df['LBDHDD_outcome'].max():.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))

# Plot Training Actuals
sns.kdeplot(y_train, label='Train Actuals (Ground Truth)', color='blue', bw_adjust=1)

# Plot Test Predictions
sns.kdeplot(preds_df['LBDHDD_outcome'], label='Test Predictions (pred.csv)', color='red', bw_adjust=1)

plt.title('Comparison of Predicted HDL Distribution vs. Training Ground Truth')
plt.xlabel('HDL-Cholesterol Value')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()